In [1]:
from backend.database import engine
from sqlmodel import Session, select, text
from sqlalchemy import func as F
from commons.schemas import *

from backend.queries import *

import pandas as pd
import numpy as np

db = Session(engine)

In [2]:

def query_per_competition(comp_id: int | None=None):
    query = f"""
        SELECT skater.first_name, skater.last_name, category.level as category_level, category.age as category_age, category.genre as category_genre, performance.rank, performance.score, performance.total_entries, club.abbrev as club, competition.name as competition_name, competition.season 
        FROM performance
        JOIN skater ON performance.skater_id = skater.id
        JOIN club ON skater.club_id = club.id
        JOIN category ON performance.category_id = category.id
        JOIN competition ON category.competition_id = competition.id
    """
    if comp_id is not None:
        query += f"\tWHERE competition.id = {comp_id}\n"
    query += "\tORDER BY category.level, category.age, category.genre, performance.rank\n"
    return text(query)

print(query_per_competition(1))


        SELECT skater.first_name, skater.last_name, category.level as category_level, category.age as category_age, category.genre as category_genre, performance.rank, performance.score, performance.total_entries, club.abbrev as club, competition.name as competition_name, competition.season 
        FROM performance
        JOIN skater ON performance.skater_id = skater.id
        JOIN club ON skater.club_id = club.id
        JOIN category ON performance.category_id = category.id
        JOIN competition ON category.competition_id = competition.id
    	WHERE competition.id = 1
	ORDER BY category.level, category.age, category.genre, performance.rank



In [3]:

# tcp_competitions = pd.read_sql(query_competitions_from_club("TOUCP"), db.connection())
# print(tcp_competitions)
perf_competitions = pd.read_sql(query_per_competition(), db.connection())
print(perf_competitions)


     first_name       last_name category_level      category_age  \
0        Samuel         VANNIER   Adulte Acier                     
1          Sara         PAPPOLA  Adulte Argent                     
2       Stefany          LEONET  Adulte Argent                     
3      Violette          CLUZEL  Adulte Argent                     
4       Stefany          LEONET  Adulte Argent                     
...         ...             ...            ...               ...   
1465      Kiara  GARCES RICHTER           R3 D  Poussin-Benjamin   
1466      Kiara  GARCES RICHTER           R3 D  Poussin-Benjamin   
1467      Kiara  GARCES RICHTER           R3 D  Poussin-Benjamin   
1468      Kiara  GARCES RICHTER           R3 D  Poussin-Benjamin   
1469      Kiara  GARCES RICHTER           R3 D  Poussin-Benjamin   

     category_genre  rank  score  total_entries   club  \
0            Hommes   1.0  12.31              1  GRCEA   
1             Dames   1.0  21.10              2  TOUCP   
2        

In [4]:
def compute_club_trophy_score(row):
    if np.isnan(row["rank"]):
        return 0, 0, 0
    podium = 0
    if row["rank"] == 1:
        podium = 3
    elif row["rank"] == 2:
        podium = 2
    elif row["rank"] == 3:
        podium = 1
    position = max(min(row["total_entries"], 10) - row["rank"] + 1,0)
    return podium, position, podium + position

perf_competitions[["trophy_podium", "trophy_position", "trophy_total"]] = perf_competitions.apply(compute_club_trophy_score, axis=1, result_type='expand')
print(perf_competitions[["first_name", "last_name", "club", "competition_name", "season", "category_level", "category_age", "category_genre", "rank", "trophy_podium", "trophy_position", "trophy_total"]].sort_values(by=["competition_name","trophy_total"]))

     first_name        last_name   club    competition_name     season  \
328      Rachel         MARTINEZ  FONTR  Coupe Gerard Prido  2023-2024   
365    Mathilde             MERY  MONTP  Coupe Gerard Prido  2023-2024   
386       Lucie          SANCHEZ  TOUCP  Coupe Gerard Prido  2023-2024   
814       Suzon          BLAZART  BRIVE  Coupe Gerard Prido  2023-2024   
935       Laura          PASCUAL  TOUCP  Coupe Gerard Prido  2023-2024   
...         ...              ...    ...                 ...        ...   
866      Sofiia         SUBOTINA  BLAGN   Trophée du Soleil  2022-2023   
943      Jeanne   BOUTON MARGAIL  FONTR   Trophée du Soleil  2023-2024   
1099      Maeva  ZALATEU--LATGER  BLAGN   Trophée du Soleil  2023-2024   
1259      Sarah           BESSAC  TOUCP   Trophée du Soleil  2023-2024   
1330       Leia         CLEMENTE  BLAGN   Trophée du Soleil  2023-2024   

     category_level                 category_age category_genre  rank  \
328         Fédéral                   

In [5]:
club_trophies = perf_competitions.groupby(
    ["season", "club", "competition_name"]
).agg(
    {
        "trophy_podium": "sum", 
        "trophy_position": "sum", 
        "trophy_total": "sum",
        "last_name": "count"
    }).rename(columns={"last_name": "total_entries"})
club_trophies["skater_ratio"] = club_trophies["trophy_total"] / club_trophies["total_entries"]
club_trophies = club_trophies.sort_values(by=["season", "competition_name","trophy_total"], ascending=False).reset_index()
print(club_trophies)

        season   club    competition_name  trophy_podium  trophy_position  \
0    2023-2024  BLAGN   Trophée du Soleil           36.0            138.0   
1    2023-2024  TOUCP   Trophée du Soleil           29.0            113.0   
2    2023-2024  MONTP   Trophée du Soleil           22.0            105.0   
3    2023-2024  MARSE   Trophée du Soleil           23.0             94.0   
4    2023-2024  NIMES   Trophée du Soleil           35.0             80.0   
..         ...    ...                 ...            ...              ...   
123  2022-2023  NIMES  Coupe Gérard Prido           27.0             54.0   
124  2022-2023  FONTR  Coupe Gérard Prido           25.0             36.0   
125  2022-2023  CASTR  Coupe Gérard Prido            5.0             15.0   
126  2022-2023         Coupe Gérard Prido            3.0              4.0   
127  2022-2023  NARBO  Coupe Gérard Prido            0.0              1.0   

     trophy_total  total_entries  skater_ratio  
0           174.0         